In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
import os
from google.colab import files

In [ ]:
PROJECT_PATH = "/content/drive/MyDrive/voice_project"
STATS_PATH = os.path.join(PROJECT_PATH, "scaling_stats")

# Model Paths
# Model A: Baseline (10k) - Used for GENDER and AGE baseline
MODEL_A_PATH = os.path.join(PROJECT_PATH, "best_voice_model.pth")

# Model B: Big Data (55k) - Used for AGE
MODEL_B_PATH = os.path.join(PROJECT_PATH, "best_age_model_50plus.pth")

# Model C: Augmented (10k) - Used for AGE
MODEL_C_PATH = os.path.join(PROJECT_PATH, "First_model_building.pth")

In [ ]:
SAMPLE_RATE = 22050
DURATION = 5
N_MELS = 128
FIXED_LENGTH = SAMPLE_RATE * DURATION
N_FFT = 2048
HOP_LENGTH = 512

In [ ]:
# Old System (6 Classes) - Used by Model A & C
AGE_MAP_6 = ['Teens', 'Twenties', 'Thirties', 'Fourties', 'Fifties', '60 Plus']
# New System (5 Classes) - Used by Model B (and for final display)
AGE_MAP_5 = ['Teens', 'Twenties', 'Thirties', 'Fourties', '50 Plus']
GENDER_MAP = ['Female', 'Male']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
try:
    GLOBAL_MEAN = np.load(os.path.join(STATS_PATH, "global_mean.npy"))
    GLOBAL_STD = np.load(os.path.join(STATS_PATH, "global_std.npy"))
    print(f"Stats Loaded: Mean={GLOBAL_MEAN:.2f}, Std={GLOBAL_STD:.2f}")
except:
    print("Stats not found. Using defaults.")
    GLOBAL_MEAN, GLOBAL_STD = -59.25, 11.95

In [ ]:
# Architecture for Model A & C (Old 6-class system)
class VoiceCNN_Old(nn.Module):
    def __init__(self):
        super(VoiceCNN_Old, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1); self.bn1 = nn.BatchNorm2d(16); self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1); self.bn2 = nn.BatchNorm2d(32); self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1); self.bn3 = nn.BatchNorm2d(64); self.pool3 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc_shared = nn.Linear(64 * 16 * 27, 128); self.fc_bn = nn.BatchNorm1d(128); self.dropout = nn.Dropout(0.5)
        self.age_head = nn.Linear(128, 6)
        self.gender_head = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(F.relu(self.fc_bn(self.fc_shared(x))))
        return self.age_head(x), self.gender_head(x)

# Architecture for Model B (New 5-class system)
class AgeCNN_New(nn.Module):
    def __init__(self):
        super(AgeCNN_New, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1); self.bn1 = nn.BatchNorm2d(32); self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1); self.bn2 = nn.BatchNorm2d(64); self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1); self.bn3 = nn.BatchNorm2d(128); self.pool3 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 16 * 27, 512); self.bn4 = nn.BatchNorm1d(512); self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(F.relu(self.bn4(self.fc1(x))))
        return self.fc2(x)

In [ ]:
model_a = VoiceCNN_Old().to(device)
if os.path.exists(MODEL_A_PATH):
    model_a.load_state_dict(torch.load(MODEL_A_PATH, map_location=device))
    model_a.eval()
    print("Model A (Baseline) Loaded")
else: print("Model A NOT FOUND")

model_b = AgeCNN_New().to(device)
if os.path.exists(MODEL_B_PATH):
    model_b.load_state_dict(torch.load(MODEL_B_PATH, map_location=device))
    model_b.eval()
    print("Model B (Big Data) Loaded")
else: print("Model B NOT FOUND")

model_c = VoiceCNN_Old().to(device)
if os.path.exists(MODEL_C_PATH):
    model_c.load_state_dict(torch.load(MODEL_C_PATH, map_location=device))
    model_c.eval()
    print("Model C (Augmented) Loaded")
else: print("Model C NOT FOUND")

In [ ]:
# --- 5. Updated Prediction Logic (Full Confidence Breakdown) ---
def predict_all():
    print("\n📂 Click below to upload your voice clip (MP3/WAV)...")
    uploaded = files.upload()

    # Define labels locally to ensure they exist
    AGE_LABELS_LIST = ['Teens', 'Twenties', 'Thirties', 'Fourties', '50 Plus']
    GENDER_LABELS_LIST = ['Female', 'Male']

    for filename in uploaded.keys():
        print(f"\n Processing {filename}...")
        try:
            # 1. Process Audio
            audio, sr = librosa.load(filename, sr=SAMPLE_RATE, duration=DURATION)
            if len(audio) < FIXED_LENGTH:
                audio = np.pad(audio, (0, FIXED_LENGTH - len(audio)), 'constant')
            else:
                audio = audio[:FIXED_LENGTH]

            spec = librosa.feature.melspectrogram(y=audio, sr=SAMPLE_RATE, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH)
            log_spec = librosa.power_to_db(spec, ref=np.max)
            norm_spec = (log_spec - GLOBAL_MEAN) / GLOBAL_STD

            tensor = torch.tensor(norm_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

            # 2. Run Inference
            with torch.no_grad():
                # Model A
                age_logits_a, gender_logits_a = model_a(tensor)
                # Model B
                age_logits_b = model_b(tensor)
                # Model C
                age_logits_c, _ = model_c(tensor)

            # 3. Decode Results

            # -- Gender (From Model A) --
            g_probs = torch.softmax(gender_logits_a, dim=1).cpu().numpy()[0]
            gender_idx = np.argmax(g_probs)
            gender_pred = GENDER_LABELS_LIST[gender_idx]
            gender_conf = g_probs[gender_idx] * 100

            # -- Age Processing Helper --
            def get_age_distribution(logits, is_old_arch=False):
                probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
                if is_old_arch:
                    # Map 6 -> 5 classes (Sum 50s + 60s)
                    p_50plus = probs[4] + probs[5]
                    probs_5 = np.array([probs[0], probs[1], probs[2], probs[3], p_50plus])
                else:
                    probs_5 = probs
                return probs_5 * 100 # Return array of percentages

            # Get full distributions
            dist_a = get_age_distribution(age_logits_a, is_old_arch=True)
            dist_b = get_age_distribution(age_logits_b, is_old_arch=False)
            dist_c = get_age_distribution(age_logits_c, is_old_arch=True)

            # 4. Print Report
            print("\n" + "="*40)
            print(f"VOICE ANALYSIS REPORT")
            print("="*40)

            # Gender Section
            bar_len = int(gender_conf / 5)
            g_bar = "|" + "█" * bar_len + " " * (20 - bar_len) + "|"
            print(f"GENDER DETECTION (Model A)")
            print(f"   Result: {gender_pred.upper()}")
            print(f"   Confidence: {g_bar} {gender_conf:.1f}%")
            print("-" * 40)

            # Age Section
            print(f"AGE CONFIDENCE BREAKDOWN")

            def print_model_stats(name, distribution):
                print(f"\n{name}")
                # Find Winner
                winner_idx = np.argmax(distribution)
                print(f"   Winner: {AGE_LABELS_LIST[winner_idx].upper()} ({distribution[winner_idx]:.1f}%)")

                # Print Bars
                for i, prob in enumerate(distribution):
                    bar_len = int(prob / 5)
                    bar = "|" + "█" * bar_len + " " * (20 - bar_len) + "|"
                    print(f"   {AGE_LABELS_LIST[i]:<10} {bar} {prob:.1f}%")

            print_model_stats("1. Model A (Baseline)", dist_a)
            print_model_stats("2. Model B (Big Data - Recommended)", dist_b)
            print_model_stats("3. Model C (Augmented)", dist_c)

            print("="*40)

        except Exception as e:
            print(f"Error: {e}")



In [ ]:
predict_all()